In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import psycopg2
import datetime
import re
import locale
import sys
from itertools import product
import time
import ipycytoscape
from ipycytoscape import CytoscapeWidget, Node, Edge

In [3]:
conn = psycopg2.connect(
    host="localhost",
    database="MF",
    user="dzukas",
    password="dzukas6",
    port=5432
) 

cur = conn.cursor()

In [ ]:
cur.execute("""
    SELECT m1.morph AS source,
           m2.morph AS target
      FROM relations AS mr
      LEFT JOIN morphs AS m1 ON mr.m1_id = m1.id
      LEFT JOIN morphs AS m2 ON mr.m2_id = m2.id
     WHERE m1.morph = %s OR m2.morph = %s;
""", ("rožė", "rožė"))
rows = cur.fetchall()

import networkx as nx
from ipycytoscape import CytoscapeWidget


# 1) Build a MultiGraph and add every edge you want,
#    tagging nodes with a 'label' and edges with 'weight' & 'relation'.
G = nx.MultiGraph()
for src, tgt in rows:
    # ensure node attributes (so CytoscapeWidget picks up labels)
    G.add_node(src, label=src)
    G.add_node(tgt, label=tgt)
    G.add_edge(src, tgt)

# 2) Push it into ipycytoscape, preserving all the parallels
cyto = CytoscapeWidget()
cyto.layout.height = '1000px'
cyto.graph.add_graph_from_networkx(
    G,
    directed=False,
    multiple_edges=True,      
)                                    


cyto.set_style([
    {
        "selector": "node",
        "style": {
            "content":          "data(label)",
            "text-valign":      "center",
            "background-color": "#61bffc"
        }
    },
    {
        "selector": "edge",
        "style": {
            "curve-style":            "bezier",
            "edge-distances":         "node-position",
            # "line-color":             "#a3c4dc"
        }
    },
    {"selector": 'edge[relation = "sn"]', "style": {"line-color": "#e41a1c"}},
    {"selector": 'edge[relation = "tn"]', "style": {"line-color": "#377eb8"}},
    {"selector": 'edge[relation = "pn"]', "style": {"line-color": "#4daf4a"}},
    {"selector": 'edge[relation = "cn"]', "style": {"line-color": "#984ea3"}},
])

cyto


In [ ]:
cur.execute("""
    SELECT morph FROM morphs WHERE morph ~ %s
""", ("",))
rows = cur.fetchall()

import networkx as nx
from ipycytoscape import CytoscapeWidget


# 1) Build a MultiGraph and add every edge you want,
#    tagging nodes with a 'label' and edges with 'weight' & 'relation'.
G = nx.MultiGraph()
for morph in rows:
    # ensure node attributes (so CytoscapeWidget picks up labels)
    G.add_node(morph, label=morph)
    # G.add_node(tgt, label=tgt)
    # G.add_edge(src, tgt)

# 2) Push it into ipycytoscape, preserving all the parallels
cyto = CytoscapeWidget()
cyto.layout.height = '1000px'
cyto.graph.add_graph_from_networkx(
    G,
    directed=False,
    multiple_edges=True,      
)                                    


cyto.set_style([
    {
        "selector": "node",
        "style": {
            "content":          "data(label)",
            "text-valign":      "center",
            "background-color": "#90e243"
        }
    },
    {
        "selector": "edge",
        "style": {
            "curve-style":            "bezier",
            "edge-distances":         "node-position",
            # "line-color":             "#a3c4dc"
        }
    },
    {"selector": 'edge[relation = "sn"]', "style": {"line-color": "#e41a1c"}},
    {"selector": 'edge[relation = "tn"]', "style": {"line-color": "#377eb8"}},
    {"selector": 'edge[relation = "pn"]', "style": {"line-color": "#4daf4a"}},
    {"selector": 'edge[relation = "cn"]', "style": {"line-color": "#984ea3"}},
])

cyto


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

In [96]:
cur.execute("""
    SELECT m1.morph AS source,
           m2.morph AS target,
           mr.sn, mr.tn, mr.pn, mr.cn
      FROM relations AS mr
      LEFT JOIN morphs AS m1 ON mr.m1_id = m1.id
      LEFT JOIN morphs AS m2 ON mr.m2_id = m2.id
     WHERE m1.morph = %s OR m2.morph = %s;
""", ("rožė", "rožė"))
rows = cur.fetchall()

import networkx as nx
from ipycytoscape import CytoscapeWidget


# 1) Build a MultiGraph and add every edge you want,
#    tagging nodes with a 'label' and edges with 'weight' & 'relation'.
G = nx.MultiGraph()
for src, tgt, sn, tn, pn, cn in rows:
    # ensure node attributes (so CytoscapeWidget picks up labels)
    G.add_node(src, label=src)
    G.add_node(tgt, label=tgt)

    for rel_type, w in (("sn", sn), ("tn", tn), ("pn", pn), ("cn", cn)):
        if w and w > 0:
            # key ensures each parallel edge is unique; you could also let NX auto-assign integer keys
            key = f"{rel_type}_{src}_{tgt}_{w}"
            G.add_edge(src, tgt, key=key, weight=w, relation=rel_type)
            # print(G.edges)

# 2) Push it into ipycytoscape, preserving all the parallels
cyto = CytoscapeWidget()
cyto.layout.height = '1000px'
cyto.graph.add_graph_from_networkx(
    G,
    directed=False,
    multiple_edges=True,            # <- crucial for MultiGraph
    # node_attrs=["label"],           # bring in our node labels
    # edge_attrs=["weight", "relation"]  # bring in weight & relation so we can style by them
)                                    

# 3) Style just as before (you can choose bundled vs unbundled Bézier)
max_weight = max(data["weight"] for _, _, _, data in G.edges(keys=True, data=True))

cyto.set_style([
    {
        "selector": "node",
        "style": {
            "content":          "data(label)",
            "text-valign":      "center",
            "background-color": "#61bffc"
        }
    },
    {
        "selector": "edge",
        "style": {
            "curve-style":            "bezier",
            "control-point-distance": f"mapData(weight, 0, {max_weight}, 20, 80)",
            "edge-distances":         "node-position",
            "width":                  f"mapData(weight, 0, {max_weight}, 0.1, 1)",
            # "line-color":             "#a3c4dc"
        }
    },
    {"selector": 'edge[relation = "sn"]', "style": {"line-color": "#e41a1c"}},
    {"selector": 'edge[relation = "tn"]', "style": {"line-color": "#377eb8"}},
    {"selector": 'edge[relation = "pn"]', "style": {"line-color": "#4daf4a"}},
    {"selector": 'edge[relation = "cn"]', "style": {"line-color": "#984ea3"}},
])

cyto


CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …

In [94]:
import networkx as nx
from pyvis.network import Network

cur.execute("""
            SELECT m1.morph, m2.morph
            FROM relations AS r
            LEFT JOIN (
            SELECT morph, id
            FROM morphs
            ) as m1
            ON r.m1_id = m1.id
            LEFT JOIN (
            SELECT morph, id
            FROM morphs
            ) as m2
            ON r.m2_id = m2.id
            --WHERE m1.morph = 'tiktaktau' or m2.morph = 'tiktaktau' or m1.morph = 'ik.*ak.*au$' or m2.morph = 'ik.*ak.*au$'
""")
edges = cur.fetchall()

G = nx.Graph()
for edge in edges:
    G.add_edge(edge[0], edge[1])

net = Network(notebook=True, cdn_resources='remote', height="100vh", width = "100%")
net.from_nx(G)
net.show("graph.html")

KeyboardInterrupt: 